## import overpy

In [1]:
import overpy
api = overpy.Overpass()

## download file from overpass api 

## or load file from disk

In [2]:
file = open("interpreter", encoding='utf-8')
result = api.parse_xml(file.read(), encoding='utf-8', parser=None)

## get all transport line, 

In [3]:
#transport = 'train'
#transport = 'bus'
transport = 'subway'

In [4]:
relation_transport= {}
for relation in result.relations:
    for member in relation.members:
        if 'route' in relation.tags.keys():
            if transport in relation.tags['route']:
                relation_transport [relation.id]= relation.tags['name']

In [5]:
for key in relation_transport:
    print (relation_transport[key])

Métro 8 : Créteil-Pointe du Lac → Balard
Métro 7: La Courneuve - 8 mai 1945 → Mairie d'Ivry
Métro 5 : Bobigny-Pablo Picasso → Place d'Italie
Métro 14 : Mairie de Saint-Ouen → Olympiades
Métro 4 : Porte de Clignancourt → Bagneux-Lucie Aubrac
Métro 10: Boulogne - Pont de Saint-Cloud → Gare d'Austerlitz
Métro 12 : Mairie d'Aubervilliers → Mairie d'Issy
Métro 9 : Pont de Sèvres → Mairie de Montreuil
Métro 1 : Château de Vincennes → La Défense (Grande Arche)
Métro 3: Pont de Levallois – Bécon → Gallieni
Métro 11: Châtelet → Mairie des Lilas
Métro 13: Saint-Denis-Université → Châtillon-Montrouge
Métro 3bis: Porte des Lilas → Gambetta
Métro 6 : Charles de Gaulle-Étoile → Nation
Métro 2 : Nation → Porte Dauphine
Métro 7 bis
Métro 4 : Bagneux-Lucie Aubrac → Porte de Clignancourt
Métro 1 : La Défense (Grande Arche) → Château de Vincennes
Métro 14 : Olympiades → Mairie de Saint-Ouen
Métro 9 : Mairie de Montreuil → Pont de Sèvres
Métro 10: Gare d'Austerlitz → Boulogne - Pont de Saint-Cloud
Métro 6

## get relations from transport id

In [6]:
transport_to_relation= {}
for relation in result.relations:
    if 'route' in relation.tags.keys():
        if transport in relation.tags['route']:
            for member in relation.members:
                if(member.role=='stop'):
                    if member.ref in transport_to_relation :
                        transport_to_relation [member.ref].append(relation.id)
                    else:
                        transport_to_relation [member.ref] = [relation.id]
                    

In [7]:
#transport_to_relation

In [8]:
for key in transport_to_relation:
    node = result.get_nodes(key)
    print (node)
    if (node):
        print (node[0].lat, node[0].lon)
    break

[]


## get relation from way id 

In [9]:
way_to_relation= {}
for relation in result.relations:
    if 'route' in relation.tags.keys():
        if transport in relation.tags['route']:
            for member in relation.members:
                if(member.role==None):
                    if member.ref in way_to_relation :
                        way_to_relation [member.ref].append(relation.id)
                    else:
                        way_to_relation [member.ref] = [relation.id]

In [10]:
#way_to_relation

##  create point layer by using Fiona

In [11]:
import fiona

In [12]:
# define schema
schema = {
    'geometry':'LineString',
    'properties':[('relation_id','int'),('relation_name','str') ,('osm_id','int')]
}

file = transport + "_line.shp"
print (file)
LineStringShp = fiona.open(file, mode='w', driver='ESRI Shapefile',schema = schema, crs = "EPSG:4326")
#iterate over each row in the dataframe and save record
for key in way_to_relation:
    try : 
        ways = result.get_way(key)
        if (ways):
            for i in way_to_relation[key]:
                line_geom = []
                nodes =  ways.get_nodes()
                if (nodes):
                    for node in nodes:
                        node_geom = (node.lon, node.lat)
                        line_geom.append (node_geom)
                #print (line_geom)
                    rowDict = {
                        'geometry' : {'type':'LineString','coordinates': line_geom },
                        'properties': {  'relation_id': i , 'relation_name' : relation_transport[i] , 'osm_id' : ways.id},
                    }
                    LineStringShp.write(rowDict)
    except:
        continue

#close fiona object
LineStringShp.close()

subway_line.shp


In [13]:
# define schema
schema = {
    'geometry':'Point',
    'properties':[('relation_id','int'),('relation_name','str') ,('osm_id','int')]
}

file = transport + "_stop.shp"
print (file)
pointShp = fiona.open(file, mode='w', driver='ESRI Shapefile',schema = schema, crs = "EPSG:4326")
#iterate over each row in the dataframe and save record
for key in transport_to_relation:
    node = result.get_nodes(key)
    if (node):
        for i in transport_to_relation[key]:
            rowDict = {
                'geometry' : {'type':'Point','coordinates': (node[0].lon,node[0].lat)},
                'properties': {  'relation_id': i , 'relation_name' : relation_transport[i] , 'osm_id' : node[0].id},
            }
            pointShp.write(rowDict)
    
#close fiona object
pointShp.close()

subway_stop.shp


## point.shp

| relation_id| relation_name | osm_id |
| --- | --- | --- |
| 1255723 | Bus 29 : Porte de Montempoivre ? Gare Saint-Lazare | 2042462222 |
| 1255729 | Bus 29 : Gare Saint-Lazare ? Porte de Montempoivre | 2042462222 |

## find the nearly building for each point, add the info to the building

hub_distance{ 'FIELD' : 'osm_id', 'HUBS' : 'batiment_converted.shp', 'INPUT' : 'point.shp', 'OUTPUT' : 'TEMPORARY_OUTPUT', 'UNIT' : 0 }

|relation_id|relation_name|osm_id|code|fclass|name|hubname|hubdist|
|---|---|---|---|---|---|---|---|
|1254454	|Bus 20 : Porte des Lilas ? Louison Bobet	|1821967163	|5621	|bus_stop	|Grands Boulevards	|5013364	|9.516719518748575|
|1255724	|Bus 39 : Gare du Nord ? Frères Voisin (Issy-les-Moulineaux)	|1821967163	|5621	|bus_stop	|Grands Boulevards	|5013364	|9.516719518748575|